SVM_CUP_GRIDS_COARSE = {
  "linear_rbf_sigmoid": {
   "estimator__kernel": ['linear', 'rbf','sigmoid'],
   "estimator__C":[1e-4,1e-1,1e1,1e4],
   "estimator__gamma" : ['scale', 'auto',1e-4,1e-1,1e1,1e4],
   "estimator__tol": [1e-4, 1e-8, 1e-2, 1e-1],
   "estimator__epsilon":[1e-4,1e-1,1e1,1e4],
   "estimator__max_iter": [10000]

  },
  "poly": {
    "estimator__kernel": ['poly'],
    "estimator__C":[1e-4,1e-1,1e1,1e4],
    "estimator__degree": [2,3,5,7,9],
    "estimator__gamma" : ['scale', 'auto', 1e-4,1e-1,1e1,1e4],
    "estimator__tol": [1e-4, 1e-8, 1e-2, 1e-1],
    "estimator__epsilon":[1e-4,1e-1,1e1,1e4],
    "estimator__max_iter": [10000],

  }
}

SVM_CUP_GRIDS_FINER = {
  "unique_entry":{
    'estimator__C': [4],
    'estimator__epsilon': [0.2],
     'estimator__gamma': [0.1],
     'estimator__kernel': ['rbf'],
     'estimator__tol': [1e-8]
  }
}


#alpha multiply L2 term must be in [0, inf), alpha=0 equiv to LinearReg and not recommended for Ridge, Lasso, Elastic, must use LinearRegression
LASSO_CUP_GRIDS_COARSE = {
    "cyclic_selec": {
        "selection": ["cyclic"],
        "alpha": [1e-4, 1e-1, 1e1, 1e4],
        "positive":[False, True]
                    },
    "random_selec": {
        "selection": ["random"],
        "alpha": [1e-4, 1e-1, 1e1, 1e4],
        "positive":[False, True],
        "random_state": [42],
             }
}

LASSO_CUP_GRIDS_FINER = {
"unique_entry":
{'alpha': [1e-4], #,1e-5,1e-7,1e-9
 'positive': [False], 
 'random_state': [42],
 'selection': ['random']}
}


RIDGE_CUP_GRIDS_COARSE = {
    "others": {
          "solver": ['saga', 'svd', 'lsqr', 'sag', 'sparse_cg', 'cholesky'],
           "alpha": [1e-4, 1e-1, 1e1, 1e4]
           },
    "lbfgs_positive": {
        "solver": ['lbfgs'], #lbfgs can be used only when positive = True
        "alpha": [1e-4, 1e-1, 1e1, 1e4],
        "positive":[True]
                    },
    "sag_saga": {
        "solver": [ 'sag', 'saga'],
        "alpha": [1e-4, 1e-1, 1e1, 1e4],
        "random_state": [42],
             }
}



RIDGE_CUP_GRIDS_FINER ={

"unique entry":{
     "solver":['saga'], #1st: 'saga','sag','svd' => 'saga'
     "alpha":[0.1,1,2,3,4,5,6,7,8,9,10], # 1st:1e-4,1e-1,1e4 1e1(winner) 2nd:1e1,1e1,1e2,1e3  3rd:1,2,3,4,5,6,7,8,9,10  => winner: 2
                },

}


ELASTICNET_CUP_GRIDS_COARSE = {
    "cyclic_selec": {
        "selection": ["cyclic"],
        "alpha": [1e1, 1e-4, 1e-1, 1e4],
        "positive":[False, True],
        "estimator__l1_ratio":[0.1,0,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0] # must be in [0,1]
                    },
    "random_selec": {
        "selection": ["random"],
        "alpha": [1e1, 1e-4, 1e-1, 1e4],
        "random_state": [42],
        "positive":[False, True],
        "l1_ratio":[0.1,0,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0] 
             }
}

ELASTICNET_CUP_GRIDS_FINER = {
 "unique_entry": {
     'alpha': [1e-4], #1e-5,1e-7,1e-9 
     'l1_ratio': [0], 
     'positive': [False], 
     'selection': ['cyclic']
     }
  }

DTR_GRID_COARSE = {
      "unique_entry":{
        # poisson criterion cannot be used, there are negative target values
        'criterion' : ['friedman_mse', 'squared_error', 'absolute_error'],
        'splitter' : ['best', 'random'],
        'max_depth': list(range(2, 12)), #max depth is #training_samples-1  
        'min_samples_split': list(range(2, 4)), 
        'min_samples_leaf': list(range(1, 4)), 
        'min_weight_fraction_leaf': [0.007,  0.008, 0.009], # 0.005 
        'ccp_alpha' : [0.0005,0.008,  0.01] #, 0.012]
    }
}


DTR_GRID_FINER = {
      "unique_entry":{

        'criterion' : [ 'squared_error'], #'friedman_mse', 'absolute_error'],
        'splitter' : ['best'], # 'random'],
        'max_depth': [7,9],   
        'min_samples_split': list(range(2, 4)),
        'min_samples_leaf': list(range(1, 4)), 
        'min_weight_fraction_leaf': [0.01], # 0.009, 0.005 
        'ccp_alpha' : [ 0.008] #,0.007, 0.009]
    }
}

RFR_GRID_COARSE = {

    "unique_entry":{
        
        'n_estimators': [30, 60, 90, 130], 
        'criterion' : ['friedman_mse', 'squared_error', 'absolute_error'],
        'max_depth' : [2,4,8], 
        'min_samples_split': [2,4], 
        'min_samples_leaf': [1,2,4,8],
        'min_weight_fraction_leaf': [0, 0.001, 0.009], 
        'ccp_alpha' : [0.001, 0.01, 0.03] 
    }
}



KNR_CUP_GRID_COARSE ={
    "unique_entry":{
     'n_neighbors': [5, 10, 15, 100,150, 200],
     'weights': ['uniform', 'distance'], 
     'p' : [1,2, 1e1, 1e3, 1e5],
     'metric': ['minkowski'], 
     'algorithm': ['ball_tree', 'kd_tree']

  }
}
KNR_CUP_GRID_FINER = {
    "unique_entry":{
     'n_neighbors': [17], # 15, 20, 25, 30],
     'weights': ['uniform'],# 'distance'], 
     'p' : [1,2],
     'metric': ['minkowski'], 
     'algorithm': ['ball_tree']#, 'kd_tree']

  }   
}

In [479]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from cup_helpers import read_ds
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from numpy import mean
from sklearn.metrics import make_scorer
import tensorflow as tf
from cup_helpers import SEED
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

#the TR/VALIDATION and your internal TEST errors, in the original scale i.e. MEE for the 2023 cup (see next slides).

In [480]:
# Datasets Path
TR_PATH = "./data/ML-CUP23-TR.csv"
TS_PATH = "./data/ML-CUP23-TS.csv"

In [481]:
(X,y) = read_ds(TR_PATH)

In [482]:
cv_strategy = KFold(n_splits=5, random_state=42, shuffle=True)

In [483]:
# Train/internal test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=SEED)

In [484]:
#custom mee metric
def mee(y_true,y_pred):
      if type(y_true) != tf.Tensor:
        y_true = np.array(y_true.astype(float))
      if type(y_pred) != tf.Tensor:
        y_pred = np.array(y_pred.astype(float))

      if y_true.ndim > 1:
          l2_norms = np.linalg.norm(np.subtract(y_pred, y_true), axis=1)
          return mean(l2_norms, axis=0)
      else:
          l2_norms = []
          for p in range(len(y_true)):
                l2_norms.append(np.linalg.norm(np.subtract(y_pred[p], y_true[p])))
          return mean(l2_norms)
      
custom_scores = {
    "mee": make_scorer(mee, greater_is_better=False),
}

In [485]:
# Function extracting each grid from dictionary of grids
def list_grids(grids_dict):
    return [grids_dict[item] for item in grids_dict]

In [486]:
GRID_SVR = {
  "linear_rbf_sigmoid": {
   "estimator__kernel": ['linear','rbf','sigmoid'],
   "estimator__C":[0.0001, 0.1, 10, 10000],
   "estimator__gamma" : ['scale', 'auto',1e-4,1e-1,1e1,1e4],
   "estimator__tol": [1e-4, 1e-8, 1e-2, 1e-1],
   "estimator__epsilon":[1e-4,1e-1,1e1,1e4],
   "estimator__max_iter": [10000]

  },
  "poly": {
    "estimator__kernel": ['poly'],
    "estimator__kernel": ['poly'],
    "estimator__C": [0.0001, 0.1, 10, 10000],
    "estimator__degree": [2, 3, 5, 7, 9],
    "estimator__gamma": ['scale', 'auto', 0.0001, 0.1, 10, 10000],
    "estimator__tol": [0.0001, 1e-8, 0.01, 0.1],
    "estimator__epsilon": [0.0001, 0.1, 10, 10000],
    "estimator__max_iter": [10000],
  }
}


GRID_LINEAR_REGRESSION = {"unique_entry": {"fit_intercept": [True]}}

GRID_LASSO = {
    "cyclic_selec": {
        "selection": ["cyclic"],
        "alpha": [0.0001, 0.1, 10.0, 10000.0],
        "positive": [False, True]
    },
    "random_selec": {
        "selection": ["random"],
        "alpha": [0.0001, 0.1, 10.0, 10000.0],
        "positive": [False, True],
        "random_state": [SEED],
    }
}

GRID_RIDGE = {
    "others": {
        "solver": ['saga', 'svd', 'lsqr', 'sag', 'sparse_cg', 'cholesky'],
        "alpha": [0.0001, 0.1, 10.0, 10000.0],
    },
    "lbfgs_positive": {
        "solver": ['lbfgs'],
        "alpha": [0.0001, 0.1, 10.0, 10000.0],
        "positive": [True]
    },
    "sag_saga": {
        "solver": ['sag', 'saga'],
        "alpha": [0.0001, 0.1, 10.0, 10000.0],
        "random_state": [SEED],
    }
}

GRID_KNR = {
     "unique_entry":{
     'n_neighbors': [5, 10, 15,20, 40, 60, 100,150, 200],
     'p' : [1,2,10,1000,100000],
     'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'], 
     'metric' : ['euclidean', 'minkowski'], 
     'weights' : ['distance', 'uniform'] 
}
}
GRID_RFR = {

    "unique_entry":{
        'n_estimators': [90,85,90,95],
        'criterion' : ['absolute_error','friedman_mse', 'squared_error'],
        'max_depth' : [8,7,8,9],  
        'min_samples_split': [4,6], 
        'min_samples_leaf': [1],
        'min_weight_fraction_leaf': [0,0.001], 
        'ccp_alpha' : [0.001, 0.002] 
    }
}

params_map = {
    "SVR": GRID_SVR,
    "linear_regression":GRID_LINEAR_REGRESSION,
    "lasso_regression":GRID_LASSO,
    "ridge_regression":GRID_RIDGE,
    "knr":GRID_KNR,
    "rfr":GRID_RFR
    }

In [487]:
def execute_gridesearch(X, y, model, model_name):
  cv = cv_strategy.split(X)
  params = params_map[model_name]
  params = list_grids(params)
  grid = GridSearchCV(model, params,scoring=custom_scores,refit="mee", cv= cv,n_jobs=-1,verbose=10,return_train_score=True).fit(X, y)
  results = pd.DataFrame(grid.cv_results_)
  print("mean validation error",abs(results.loc[grid.best_index_,"mean_test_mee"]))
  print("std validation error",abs(results.loc[grid.best_index_,"std_test_mee"]))
  print("mean",abs(results.loc[grid.best_index_,"mean_train_mee"]))
  print("std",abs(results.loc[grid.best_index_,"std_train_mee"]))
  return grid



In [488]:

def do_sklearn_GridSearchCV(X,y,X_test,y_test,model,model_name):
    print("Model Used: " + model_name)
    grid = execute_gridesearch(X, y, model, model_name)
    print("Model used: " + model_name + ", best parameters: " + str(grid.best_params_) )
    y_pred = grid.best_estimator_.predict(X_test)
    print("Test MEE:", mee(y_test, y_pred))
    print("------------------------------------------------------------------------------------------------------")


In [489]:
def grid_search_model(model,model_name,X_train,y_train,X_test,y_test):
    do_sklearn_GridSearchCV(X_train,y_train,X_test,y_test,model,model_name)

# SVR

In [490]:
grid_search_model(MultiOutputRegressor(SVR()),"SVR",X_train,y_train,X_test,y_test)

Model Used: SVR
Fitting 5 folds for each of 3072 candidates, totalling 15360 fits


KeyboardInterrupt: 

# Linear Regression

## Linear regression without regularization

In [491]:
grid_search_model(LinearRegression(),"linear_regression",X_train,y_train,X_test,y_test)

Model Used: linear_regression
Fitting 5 folds for each of 1 candidates, totalling 5 fits
mean validation error 6.276929390627119
std validation error 0.19637979115352105
mean 6.0782381235331835
std 0.052920032699334055
Model used: linear_regression, best parameters: {'fit_intercept': True}
Test MEE: 6.259866063110195
------------------------------------------------------------------------------------------------------


## Linear regression with L1 regularization

In [475]:
grid_search_model(Lasso(),"lasso_regression",X_train,y_train,X_test,y_test)

## Linear regression with L2 regularization (Thikonov regularization)

In [476]:
grid_search_model(Ridge(),"ridge_regression",X_train,y_train,X_test,y_test)

# KNR

In [477]:
grid_search_model(KNeighborsRegressor(),"knr",X_train,y_train,X_test,y_test)

# RFR CV

In [478]:
grid_search_model(RandomForestRegressor(),"knr",X_train,y_train,X_test,y_test)

Model Used: knr
Fitting 5 folds for each of 720 candidates, totalling 3600 fits


KeyboardInterrupt: 